In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader, Subset
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [3]:
from modules.local_values import datas, target_datas

class CDataset(Dataset):
    def __init__(self, x, y, seq_length):
        super(CDataset, self).__init__()
        self.s_len = seq_length
    
    ### x data
        self.x = pd.read_csv(x,parse_dates=[0],index_col=[0])
        self.x = torch.FloatTensor(self.x[60:-80].T.values).to(device)
        self.len = len(self.x[0])-self.s_len

    ### y data
        self.y = pd.read_csv(y,parse_dates=[0],index_col=[0])
        self.y = torch.FloatTensor(self.y[60:-80].T.values).to(device)

    def __getitem__(self, index):
        self.x_data = self.x[:,index:index+self.s_len]
        self.y_data = self.y[:,index:index+self.s_len]
        return self.x_data, self.y_data
    
    def __len__(self):
        return self.len

dataset = CDataset(datas[0], target_datas[0], 64)
len(dataset)

3255670

In [4]:
test_size = 262144
train_set = Subset(dataset, range(len(dataset)-test_size))
test_set = Subset(dataset, range(len(dataset)-test_size,len(dataset)))

train_dloader = DataLoader(train_set, batch_size=256, shuffle=False, drop_last=True)
test_dloader = DataLoader(test_set, batch_size=128, shuffle=False, drop_last=True)

In [7]:
train_features, train_labels = next(iter(train_dloader))
print(train_features.shape, train_labels.shape)

torch.Size([256, 11, 64]) torch.Size([256, 6, 64])
